# Using the Pandas Python Package

## The example data set
This data set was downloaded from https://archive.ics.uci.edu/ml/datasets/Ozone+Level+Detection

## Importing data from text files into pandas data structures
The data is spread across multiple individual files. We'll be using two files for this example:
- eighthr.data contains the data but without any column headers
- eighthr.names containers the column headers (1 header/row) for the data file

So we need to read both files separately and add the imported column headers to the pandas data structure when importing the data.

In [ ]:
import pandas as pd

# read file with column headers
datainfo = pd.read_csv(
    'eighthr.names',             # name of txt file to open
    header=None,                     # this file does not have any column headers
    skiprows=range(2),               # skip first two rows; they contain additional annotations that we dont need
    names=['column name','data type'],    # set column labels
    delimiter=':')                   # the ':' separates the column name from the data type in each row

# fix column labels by adding missing label 'Ozone day' to columnnames
columnnames = list(datainfo['column name'])
columnnames.append('Ozone day')
print(columnnames)

# read data file and set column names
data = pd.read_csv(
    'eighthr.data',              # name of txt file to open
    header=None,                     # this file does not have any column headers
    names=columnnames,               # set column names
    delimiter=',')                   # the ',' separates the column name from the data type in each row


## Getting a basic overview of the data

The describe() function provides simple summary statistics about the underlying DataFrame object. For this particular unprocessed data set only rows 1-3 are meaningful, but at least we can get a quick idea of the total number of columns and number of values per data row (count, or # number of instances).

In [ ]:
data.describe(include='all')

To get an idea regarding the single row values, we can use the .head() or .tail() functions, 
which output the first five or last five data rows respectively. 

In [ ]:
# print column labels and first five rows
data.head()

## Referencing and inspecting a specific column
Columns can be referenced via the column label or column index. If a single column is selected, the returned data type is a 1-dimensional DataSeries rather than a multi-dimensional DataFrame. We can store the returned data subset (selected column) in a variable.

Here are three different approaches that all return the same column data as a DataSeries:

In [ ]:
# select by column name
precipitation = data['Precp']
precipitation.head()

In [ ]:
# select by column index; here we select column with index=72
precipitation = data[data.columns[72]]
precipitation.head()

In [ ]:
# select by column index; here we select the second to last columnn (the last column has the index=-1)
precipitation = data[data.columns[-2]]
precipitation.head()

## Selecting multiple columns and assigning them to a new variable
We can use a list of column labels or column indices to extract a subset of the columns from the data set and store the subset in a new variable. When we select multiple columns, the selected data is returned as a multi-dimensional DataFrame.

The order of the list elements determines the oder of the columns in the returned DataFrame.

In [ ]:
# Select columns with label 'Ozone day' and 'Precp'
subset = data[['Date','Ozone day','Precp']]
subset.head()

In [ ]:
# Select columns by index
subset = data.iloc[:,[0,73,72]]
subset.head()

Syntax for indexing a range of columns [startindex:endindex:direction]. Following Python list/array indexing and slicing convention, the endindex is excluded. direction=-1 enables reverse order, direction=1 indicates forward index order and can be ommitted.

In [ ]:
# Select the last (index=72) and second to last (index=71) column in reverse order
subset = data[data.columns[73:71:-1]]
subset.head()

# ALTERNATIVE: use negative index (interpreted relative to end)
# select the last (-1) and second to last (-2) column.
subset = data[data.columns[-1:-3:-1]]
subset.head()

## Common Preprocessing Steps
### Dropping Columns
Sometimes you find that not all columns are required for the data analysis. 
You can remove the unwanted columns with the drop() method.

In [ ]:
# Dropping multiple columns by column label
subset = data.drop(['SLP_','RH50'],axis=1)
print('Original columns:\n',data.columns.values)
print('\nRemaining columns after drop:\n',subset.columns.values)

### Converting Strings to Dates (datetime64 objects)
The first column of our original data set contains the date as a string. We can convert the string representation 
to a datetime64 object, which will make selecting and grouping row data by specific date values (e.g. selection of 
first of each month, grouping by months or years) much easier (see section "Grouping Data based on Datetime object values").

Note the last line of the output indicating that the data type (dtype) is now datetime64.

In [ ]:
print('Before conversion\n',data['Date'].head())
data['Date'] = data['Date'].apply(pd.to_datetime, format="%m/%d/%Y")
print('\nAfter conversion\n',data['Date'].head())


### Converting Values to Numbers (float)
The first column of our original data set contains the data as a string (before we converted it to datetime objects). All other columns contain numeric values or '?' in case a particular value is missing. Due to the presence of the '?' the data type of columns 2 to 74 is a generic object. This precludes performing any mathematical oeprations on these values, like caluclating the mean or standard deviation for a group of rows. 

Let's fix that by selecting columns 2-73 (index 1 to the end) and forcing the conversion of all values in these columns to numbers. For values for which the conversion fails, the values will be set to 'NaN'. Note the replacement of '?' with 'NaN' in the last row of the data.head() output.

'NaN' values will be excluded from any mathematical operation.

In [ ]:
# Convert the column values in place
# Apply the pandas.to_numeric function to selected columns and reassign the converted columns to the original data.
data[data.columns[1:]] = data[data.columns[1:]].apply(pd.to_numeric,errors='coerce')
data.head()


Let's look at the output produced by data.decribe(). Note that we do not need to pass the argument "include='all'".

Also note that the values for 'count' vary by column now, since count only considers column values that are not 'NaN'.

In [ ]:
data.describe()

### Removing data with missing values (NaN)
After our conversion of the data types to a numeric type, we created NaN values in specif rows and/or columns. Sometimes you may wish to exclude the entire row/column with any NaN values, sometimes you may want to eliminate only those where an entire row/columns shows NaN values.  

Pandas offers a convenient function for elminating rows or columns with NaN values.

In [ ]:
# Drop the columns where all values are NaN
cleaned_data = data.dropna(axis=1, how='all')

# Drop the columns where any value is NaN
cleaned_data = data.dropna(axis=1, how='any')

# Drop the rows where all of the values are NaN
cleaned_data = data.dropna(axis=0, how='all')

# Drop the rows where any of the values are NaN
cleaned_data = data.dropna(axis=0, how='any')

# Keep only the rows with at least 2 non-NaN values:
cleaned_data = data.dropna(thresh=2)

### Converting columns with 1/0 values to boolean type
Note that the 'Ozone day' column contains the values 1 or 0, indicating whether the particular day is classified as an ozone day or not. We can convert the data of this column into the corresponding boolean values 'True' or 'False'.

In [ ]:
data['Ozone day'] = data['Ozone day'].astype('bool')
data.head()

## Filtering rows based on values in specific columns
The data can be filtered based on a set of conditions. Different conditions can be applied to different columns. If multiple conditions are used, the individual conditions need to be enclosed in ( ).

The returned DataFrame may have a different shape (number of rows) than the original DataFrame.

In [ ]:
# get data rows where the 'Precp' value is > 10 and the 'WSR0' value is < 3.
filtered_data = data[(data['Precp'] > 10) & (data['WSR0'] < 3)]
filtered_data.head()

## Data filtering and value masking with where()
The where function allows masking of row values based on a set of conditions. 
There are two key differences to the filtering approach described above:
- The returned data frame has the same shape as the input.
- For rows where the specified condition evaluates to False, the original value is replaces by NaN or NaT, 
or a specified default value.

In [ ]:
filtered_data = data.where(data['WSR0'] < 3)
filtered_data.head()

Note that the values in row 3 have been replaced with NaT ('Date' column) or NaN since the original data in this row does not meet the specified condition (data['WSR0'] < 3).

Instead of having those rows be filled with 'NaN'/'NaT', we can also specify a value to replace the original value in cases where the condition evaluates to False.

In [ ]:
# To keep it simple, we're dropping the 'Date' column since setting 'Date' to -1 does not make much sense here
filtered_data = data.drop(['Date'], axis=1).where(data['WSR0'] < 3,-1)
filtered_data.head()

## Grouping rows by values in specific column
In the example data, the 'Ozone day' column indicates wheter the specific day was considered a ozone day.

Let's group the data rows based on whether they represent ozone days or not, and calculate the mean and std for the various measured wind speeds in each group.

In [ ]:
# Create list with column labels to groupby. We use a single column, but this can be a list with multiple column labels
groupnames = ['Ozone day']

# Create list with columns to select (cnames) which contains the data columns of interest and the column(s) that the 
# data should be grouped by
datacolumns = ['Date','WSR0','WSR1','WSR2','WSR3','WSR4','WSR5']
cnames = list(groupnames)
cnames.extend(datacolumns)

# Select columns based on cnames, create a group for each value found in the groupnames column(s)
windspeed_by_ozone = data[cnames].groupby(groupnames)

# In this case we have two groups: 'True' with data for ozone days, and 'False' witb data for non-ozone days.
# Iterate over grouped data
for name,group in windspeed_by_ozone:
    print('Group name:',name,'\n')
    print('Data:\n',group,'\n')


### Grouping Data based on Datetime object values

The first column in our data provides a Datetime object. It provides convenient attributes and functions to filter data by time/date or group data by periods, e.g. days, months, years.

In [ ]:
# grouping by month but combining all years --> 12 groups shown in rows
data_by_calmonth = data.groupby(data['Date'].dt.month)
data_by_calmonth.describe()

In [ ]:
# grouping by month for each calendar year
data_by_date = data.groupby(data['Date'].dt.to_period('M'))
data_by_date.describe()

### Grouping by Multiple Columns
We can specify multiple columns to be used as keys to group the data. The number of groups corresponds to the existing unique combination of value pairs in the specified columns.

In [ ]:
# Let's group the data by year (based on 'Date' column) and 'Ozone day'
# Since we have 7 years and 2 'Ozone day' categories the data will be divided into 14 groups
multi_group_data = data.groupby([data['Date'].dt.year,'Ozone day'])
# Iterate over grouped data
for name,group in multi_group_data:
    print('Group name:',name)

multi_group_data.describe()

## Calculating Aggregate Numbers (mean, standard deviation, min, max, etc.)
Now that we have grouped the wind speed data based on the 'Ozone day' values, we can easily calculate 
some summary values for each data column in each group. Pandas DataFrames provide functions for easy calculation of many aggregate values, including:
- count
- mean
- median
- stdev
- sum
- min
- max

... and many more

In [ ]:
# Calculate means and std for each group
ws_means = windspeed_by_ozone.mean()
ws_stds = windspeed_by_ozone.std()

ws_means.head()

In the approach above, we store mean and standard deviation values in separate variables. Often it is convenient to collect the output of multiple aggregation functions in a single DataFrame. This can be done with the aggregate() function.

In [ ]:
# Calculate mean, std, min, max, count for each group and store aggregate values in single DataFrame
ws_summary = windspeed_by_ozone.aggregate(['mean','std','min','max','count'])
ws_summary.head()

## Plotting Data
There are seral ways to plot data contained in a pandas DataFrame or DataSeries.
### Using the pandas plotting functionality
Simple plots can be created by calling plot.<PLOT_TYPE>() on a pandas DataFrame or group object. These functions/methods use the matplotlib package.

Different plot types are provided by:
- pandas.DataFrame.plot.area
- pandas.DataFrame.plot.bar
- pandas.DataFrame.plot.barh
- pandas.DataFrame.plot.box
- pandas.DataFrame.plot.density
- pandas.DataFrame.plot.hexbin
- pandas.DataFrame.plot.hist
- pandas.DataFrame.plot.kde
- pandas.DataFrame.plot.line
- pandas.DataFrame.plot.pie
- pandas.DataFrame.plot.scatter
- pandas.DataFrame.boxplot
- pandas.DataFrame.hist

In [ ]:
%matplotlib inline
# Plotting the wind speed means by 'Ozone day' group
ws_means.plot.bar()

You may want to show the two group values for a given wind speed column next to each other. 
That's easy, just transpose the values before plotting

In [ ]:
ws_means_trans = ws_means.transpose()
ws_means_trans.plot.bar()

### Using matplotlib directly

In [ ]:
# The following line is needed to show plot inline in notebook
%matplotlib inline 

# Create a simple bar plot of means and std
ws_stds_trans = ws_stds.transpose()
ws_means_trans.plot.bar(yerr=ws_stds_trans)

### Scatter Plots

Sometimes you want to get a quick glance at what data columns may or may not be correlated. 
We can use scatter_matrix for this. It expects a DataFrame and creates a grid of x-y scatter plots 
of all possible two column combinations.

In [ ]:
# The following line is needed to show the plot inline in this notebook.
%matplotlib inline

from pandas.plotting import scatter_matrix

# here we use a few columns of the ungrouped original data
scatter_matrix(data[['WSR0','WSR1','WSR2','WSR3',]], figsize=(10,10))

Let's create a set of new scatter plots that displays 'WSR9' and 'Precp' grouped by year (based on 'Date' column values). Each plot will show the data for a single year grouped by 'Ozone day'. Here we use the matplotlib package directly.

In [ ]:
# The following line is needed to show the plot inline in this notebook.
%matplotlib inline

# import package for plotting
import matplotlib.pyplot as plt

groups = [data['Date'].dt.year,'Ozone day']
x = 'WSR9'
y = 'Precp'
grouped_data = data.groupby(groups)['Date','Ozone day',x,y]
maxx = data[x].max()*1.2
maxy = data[y].max()*1.2

fig, ax = plt.subplots(ncols=1,nrows=(len(grouped_data.groups)//2),figsize=(5,20))
i = 0
# create a plot for each year, each plot overlaying data for ozone days (True) and non-ozone days (False)
for name, group in grouped_data:
    row = i//2 # plotting two groups in each subplot
    ax[row].plot(group[x], group[y], marker='o', linestyle='', ms=5, label=name)
    ax[row].legend()
    ax[row].set_xlim(0,maxx)
    ax[row].set_ylim(0,maxy)
    ax[row].set_xlabel(x)
    ax[row].set_ylabel(y)
    i+=1
plt.tight_layout()    
plt.show()    